<a href="https://colab.research.google.com/github/LuisHiram99/trabajos_PLN/blob/main/Rest-Mex/Extender_vectores.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<h1>Rest Mex 2025</h1>
<h3>Vectores mixtos: concatenando embeddings y conteos</h3>

**Actividad por equipo**. Cada equipo debe presentar 3 diferentes experimentos:

1. Clasificación usando los vectores aumentados, usando el preprocesamiento y estrategia de balanceo que han estado usando en los experimentos, reportar el score F1. Para la parte de embeddings usar `doc2vec`.

2. Clasificación usando los vectores aumentados, usando el preprocesamiento y estrategia de balanceo que han estado usando en los experimentos, reportar el score F1. Para la parte de embeddings usar promedios de `word2vec`/`fasttext`/`glove`.

3. En el modelo que haya funcionado mejor de los dos anteriores, presentar otro experimento variando la longitud del vector de conteos tf-idf.

Es posible que se tengan que hacer estos *downgrades*

In [ ]:
!pip uninstall -y numpy scipy gensim
!pip install numpy==1.24.3
!pip install scipy==1.10.1
!pip install gensim==4.3.1

Found existing installation: numpy 2.0.2
Uninstalling numpy-2.0.2:
  Successfully uninstalled numpy-2.0.2
Found existing installation: scipy 1.14.1
Uninstalling scipy-1.14.1:
  Successfully uninstalled scipy-1.14.1
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 17.3/17.3 MB 24.2 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
hyperopt 0.2.7 requires scipy, which is not installed.
sklearn-pandas 2.2.0 requires scipy>=1.5.1, which is not installed.
pytensor 2.30.3 requires scipy<2,>=1, which is not installed.
yellowbrick 1.5 requires scipy>=1.0.0, which is not installed.
hdbscan 0.8.40 requires scipy>=1.0, which is not installed.
mlxtend 0.23.4 requires scipy>=1.2.1, which is not installed.
mizani 0.13.3 requires scipy>=1.8.0, which is not installed.
shap 0.47.1 requires scipy, which is not installed.
fastai 2.7.19 requires scipy, which is not insta

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.9/58.9 kB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.1/34.1 MB 52.6 MB/s eta 0:00:00
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cvxpy 1.6.5 requires scipy>=1.11.0, but you have scipy 1.10.1 which is incompatible.
albumentations 2.0.5 requires numpy>=1.24.4, but you have numpy 1.24.3 which is incompatible.
scikit-image 0.25.2 requires scipy>=1.11.4, but you have scipy 1.10.1 which is incompatible.
jax 0.5.2 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jax 0.5.2 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
jaxlib 0.5.1 requires numpy>=1.25, but you have numpy 1.24.3 which is incompatible.
jaxlib 0.5.1 requires scipy>=1.11.1, but you have scipy 1.10.1 which is incompatible.
pymc 5.21.2 requires numpy>=1.25.0, but you have numpy 1.24.

Más información: https://sites.google.com/cimat.mx/rest-mex-2025/home

In [ ]:
import pandas as pd

url = "https://raw.githubusercontent.com/gmauricio-toledo/NLP-LCC/main/Rest-Mex/Rest-Mex_2025_train.csv"
df = pd.read_csv(url)
df.head()

Title  \
0              Mi Lugar Favorito!!!!   
1  lugares interesantes para visitar   
2             No es el mismo Dreams    
3  un buen panorama cerca de CancÃºn   
4                           El mejor   

                                              Review  Polarity          Town  \
0  Excelente lugar para comer y pasar una buena n...       5.0      Sayulita   
1  andar mucho, así que un poco difícil para pers...       4.0         Tulum   
2  Es nuestra cuarta visita a Dreams Tulum, elegi...       3.0         Tulum   
3  Estando en CancÃºn, fuimos al puerto y tomamos...       4.0  Isla_Mujeres   
4  Es un lugar antiguo y por eso me encanto tiene...       5.0     Patzcuaro   

        Region        Type  
0      Nayarit  Restaurant  
1  QuintanaRoo  Attractive  
2  QuintanaRoo       Hotel  
3  QuintanaRoo  Attractive  
4    Michoacan       Hotel

#1. Polaridad

In [ ]:
df1 = df[['Title','Review','Region','Polarity']]
df1

Title  \
0                       Mi Lugar Favorito!!!!   
1           lugares interesantes para visitar   
2                      No es el mismo Dreams    
3           un buen panorama cerca de CancÃºn   
4                                    El mejor   
...                                       ...   
208046  excelente ambiente, comida y atencion   
208047                       Muy mal servicio   
208048                              Excelente   
208049                        Visita nocturna   
208050                          Un Tepoz debe   

                                                   Review            Region  \
0       Excelente lugar para comer y pasar una buena n...           Nayarit   
1       andar mucho, así que un poco difícil para pers...       QuintanaRoo   
2       Es nuestra cuarta visita a Dreams Tulum, elegi...       QuintanaRoo   
3       Estando en CancÃºn, fuimos al puerto y tomamos...       QuintanaRoo   
4       Es un lugar antiguo y por eso me encanto tiene...         Michoacan   
...                                                   ...               ...   
208046  Excelente Restaurante, comida Mexicana de alto...         Queretaro   
208047  Estuvimos allí como huéspedes de un día a la p...       QuintanaRoo   
208048  Excelente comida, así como la atención y servi...  Estado_de_Mexico   
208049  Nos toco visitar este sitio cuando ya había ca...         Chihuahua   
208050  Si dejar la ciudad sin probar tepoznieves, es ...           Morelos   

        Polarity  
0            5.0  
1            4.0  
2            3.0  
3            4.0  
4            5.0  
...          ...  
208046       5.0  
208047       2.0  
208048       5.0  
208049       4.0  
208050       5.0  

[208051 rows x 4 columns]

In [ ]:
def check_data(df,column):
    # Viendo si hay valores nulos
    print(f"nulos: \n{df.isnull().sum()}")
    # Revisando valores unicos de sentiment
    print("---------")
    #viendo la longitud de string mas pequeña en review
    print(df[column].str.len().min())
    print("---------")
    #viendo la mas larga
    print(df[column].str.len().max())
    print("---------")
    #Viendo el promedio
    print(df[column].str.len().mean())
    print("---------")
    #viendo valores unicos para review
    print(f"Valores repetidos: {df.duplicated().sum()}")
    print("---------")

In [ ]:
check_data(df1,'Review')

nulos: 
Title       2
Review      0
Region      0
Polarity    0
dtype: int64
---------
33
---------
8247
---------
357.43357157620005
---------
Valores repetidos: 180
---------


In [ ]:
df1["title_and_text"] = df1['Title'].fillna("") + " " + df1["Review"]
df1 = df1.drop(columns=['Title','Review','Region'], axis=1)
df1.drop_duplicates(subset=['title_and_text'], inplace=True)
df1.head()

<ipython-input-8-516d5bf536d8>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df1["title_and_text"] = df1['Title'].fillna("") + " " + df1["Review"]


Polarity                                     title_and_text
0       5.0  Mi Lugar Favorito!!!! Excelente lugar para com...
1       4.0  lugares interesantes para visitar andar mucho,...
2       3.0  No es el mismo Dreams  Es nuestra cuarta visit...
3       4.0  un buen panorama cerca de CancÃºn Estando en C...
4       5.0  El mejor Es un lugar antiguo y por eso me enca...

In [ ]:
check_data(df1,'title_and_text')

nulos: 
Polarity          0
title_and_text    0
dtype: int64
---------
54
---------
8299
---------
383.3040443351887
---------
Valores repetidos: 0
---------


In [ ]:
import re
import nltk
import string
import spacy
from nltk import word_tokenize
from nltk import word_tokenize
from string import punctuation

nltk.download('punkt_tab')
nltk.download('stopwords')

stopwords_list = nltk.corpus.stopwords.words('spanish')
punct_set = set(punctuation)

def clean_document(df):
    doc = df.split()
    doc = [word.lower() for word in doc if word not in (stopwords_list) and word.isalpha()]
    doc = ' '.join(doc)
    doc = doc.translate(str.maketrans('', '', string.punctuation))


    return doc


[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import accuracy_score, classification_report


X = df1['title_and_text'].values

y = df1['Polarity']
label_encoder = LabelEncoder()
y = label_encoder.fit_transform(y)

X_train, X_test, y_train, y_test = train_test_split(X, y.astype(int),
                                                    test_size=0.2,
                                                    random_state=2025)



In [ ]:
train_docs = [clean_document(doc) for doc in X_train]
test_docs = [clean_document(doc) for doc in X_test]

train_docs[0]

'día san cristobal casas excelente ubicación un lugar obligatorio a solo minuto parque experiencia diseño detalle cada las habitaciones cómodas'

In [ ]:
vectorizer = TfidfVectorizer()
X_train_tfidf = vectorizer.fit_transform(train_docs)
X_test_tfidf = vectorizer.transform(test_docs)

In [ ]:
import numpy as np

vocab = vectorizer.get_feature_names_out()
important_words = []

num_palabras =  100 # DEFINIR CUANTAS PALABRAS POR ETIQUETA SE TOMAN

for label in np.unique(y_train):
    X_j = X_train_tfidf[y_train==label].copy()
    normas = np.sum(np.abs(X_j), axis=0)
    word_normas = [(word, norma) for word, norma in zip(vocab, normas.tolist()[0])]
    word_normas = sorted(word_normas, key=lambda x: x[1], reverse=True)[:num_palabras]
    print(f"Label: {label}")
    for word, norma in word_normas:
        print(word,end=' ')
    print()
    important_words.extend([word for word, norma in word_normas])

Label: 0
no servicio pésimo mal hotel mala el comida lugar si peor la habitación solo pésima nunca después restaurante personal dijeron experiencia dos pedimos horrible agua atención pesimo ser minutos en llegamos caro mesa muy hora habitaciones así nos tan meseros nadie día recepción personas noche dijo dieron hacer calidad ir mesero tiempo vez comer es malo gente me precio lo tardaron los verdad pena pedí pesos las ver sólo terrible pagar recomiendo menos gerente mejor llegar persona al dinero actitud playa vale mesas mas falta hace por sabor veces cuando pedir parece demasiado de precios parte bien baño buena siquiera 
Label: 1
no el servicio comida hotel la mal lugar si mala habitación solo restaurante caro personal ser habitaciones agua playa mejor buena en precio así las bien dos bastante sólo tan los pedimos calidad demasiado atención después falta vez lo nos noche ver es me gente muy aunque pésimo nunca ir vista sabor recepción día puede malo experiencia mas bueno parece hace b

In [ ]:
important_words = list(set(important_words))
print(important_words)
important_words_idxs = [vocab.tolist().index(word) for word in important_words]
print(important_words_idxs)

['es', 'delicioso', 'mesero', 'restaurantes', 'dieron', 'tardaron', 'mesa', 'pedir', 'hermosa', 'buen', 'gente', 'verdad', 'menos', 'horrible', 'ir', 'único', 'amable', 'mejor', 'aquí', 'puede', 'dos', 'parecía', 'encantó', 'tan', 'este', 'mal', 'precio', 'personal', 'interesante', 'calidad', 'dijo', 'hacer', 'terrible', 'la', 'cosas', 'nadie', 'playa', 'casa', 'siquiera', 'agradable', 'no', 'creo', 'así', 'llegar', 'realmente', 'deja', 'un', 'dinero', 'mesas', 'pesimo', 'al', 'cenote', 'justo', 'pésimo', 'mi', 'muy', 'tener', 'duda', 'mar', 'casi', 'mejores', 'buena', 'ser', 'una', 'cada', 'habitación', 'comida', 'ver', 'excelente', 'parece', 'ciudad', 'me', 'peor', 'sin', 'familia', 'nunca', 'lo', 'parte', 'malo', 'falta', 'mejorar', 'llegamos', 'minutos', 'experiencia', 'esperaba', 'hace', 'solo', 'actitud', 'habitaciones', 'hermoso', 'visitar', 'los', 'vez', 'cuenta', 'pero', 'súper', 'pedimos', 'increíble', 'si', 'desayuno', 'día', 'mucha', 'mas', 'podría', 'vale', 'tulum', 'perso

In [ ]:
extra_train_features = X_train_tfidf[:, important_words_idxs].toarray()
print(extra_train_features.shape)
extra_test_features = X_test_tfidf[:, important_words_idxs].toarray()
print(extra_test_features.shape)

(166296, 187)
(41575, 187)


In [ ]:
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from nltk.tokenize import word_tokenize

tokenized_train_docs = [word_tokenize(doc) for doc in train_docs]
tokenized_test_docs = [word_tokenize(doc) for doc in test_docs]

tagged_data = [TaggedDocument(words=doc, tags=[str(i)])
               for i, doc in enumerate(tokenized_train_docs)]

# Parametros
vector_size = 200   # Dimensionality of the feature vectors
window = 5        # Maximum distance between current and predicted word within a sentence
min_count = 5     # Ignores all words with total frequency lower than this
workers = -1       # Use `workers` number of worker threads to train the model
epochs = 5       # Number of iterations (epochs) over the corpus

d2v_model = Doc2Vec(vector_size=vector_size,
                 window=window,
                 min_count=min_count,
                 workers=workers)

d2v_model.build_vocab(tagged_data)

# Entrenamiento
d2v_model.train(tagged_data, total_examples=d2v_model.corpus_count, epochs=epochs)

In [ ]:
train_embeddings = d2v_model.dv.vectors
test_embeddings = np.array([d2v_model.infer_vector(doc) for doc in tokenized_test_docs])

augmented_train_features = np.concatenate((train_embeddings, extra_train_features), axis=1) # Esta es la matriz de vectores aumentados
print(augmented_train_features.shape,train_embeddings.shape,extra_train_features.shape)
augmented_test_features = np.concatenate((test_embeddings, extra_test_features), axis=1)
print(augmented_test_features.shape,test_embeddings.shape,extra_test_features.shape)

(166296, 387) (166296, 200) (166296, 187)
(41575, 387) (41575, 200) (41575, 187)


Ya puedes hacer el entrenamiento y evaluación con:

* Vectores aumentados: `augmented_train_features`
* Vectores sin aumentar (word2vec/glove/fasttext/doc2vec) `train_embeddings`

In [ ]:
train_embeddings

array([[-2.6154078e-03, -2.9895634e-03, -4.9403771e-03, ...,
        -2.0315601e-03,  1.0761458e-03,  2.6605588e-03],
       [-3.9372956e-03, -4.6021976e-03,  4.2071347e-03, ...,
         4.7651022e-03,  6.5459847e-04, -1.2241363e-03],
       [-3.4059130e-03, -3.5879929e-03, -2.2946196e-03, ...,
        -2.4547994e-03, -2.1731847e-03,  2.2390049e-03],
       ...,
       [-4.3996191e-03,  3.6782306e-03,  4.5030136e-03, ...,
        -4.9243211e-03, -1.7822337e-03,  2.0192231e-03],
       [ 3.7238575e-03, -8.4888341e-04,  2.9521191e-03, ...,
        -3.4976529e-03, -3.3160644e-03, -3.9599258e-03],
       [-1.9971591e-03,  2.4339557e-03,  3.2996410e-03, ...,
         1.3155622e-03, -1.0104775e-05, -2.2605462e-03]], dtype=float32)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

clf = RandomForestClassifier(random_state=42)
clf.fit(augmented_train_features, y_train)

RandomForestClassifier(random_state=42)

In [ ]:
y_pred = clf.predict(augmented_test_features)

In [ ]:
from sklearn.metrics import f1_score
f1_rf = f1_score(y_test, y_pred, average='weighted')
print("F1 Score (Random Forest - Weighted):", f1_rf)

F1 Score (Random Forest - Weighted): 0.5617439940153827


In [ ]:
import pickle

with open('rfmodel.pkl','wb') as f:
    pickle.dump(clf,f)

In [ ]:
''' from sklearn.ensemble import GradientBoostingClassifier

# Entrenar y predecir con Gradient Boosting
clf_gb = GradientBoostingClassifier(random_state=2025)
clf_gb.fit(augmented_train_features, y_train)
y_pred_gb = clf_gb.predict(augmented_test_features)

# F1 Score ponderado
f1_gb = f1_score(y_test, y_pred_gb, average='weighted')
print("F1 Score (Gradient Boosting - weighted):", f1_gb) '''

' from sklearn.ensemble import GradientBoostingClassifier\n\n# Entrenar y predecir con Gradient Boosting\nclf_gb = GradientBoostingClassifier(random_state=2025)\nclf_gb.fit(augmented_train_features, y_train)\ny_pred_gb = clf_gb.predict(augmented_test_features)\n\n# F1 Score ponderado\nf1_gb = f1_score(y_test, y_pred_gb, average=\'weighted\')\nprint("F1 Score (Gradient Boosting - weighted):", f1_gb) '